In [1]:
import numpy as np
import scanpy as sc
import yaml
from pathlib import Path
from scripts.EGGFM.eggfm import run_eggfm_dimred
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score


def compute_ari(X, labels, k):
    km = KMeans(n_clusters=len(np.unique(labels)), n_init=10)
    km.fit(X[:, :k])
    return adjusted_rand_score(labels, km.labels_)


def main():
    params = yaml.safe_load(Path("configs/params.yml").read_text())
    spec = params["spec"]
    k = spec.get("ari_n_dims", spec.get("n_pcs", 10))

    base = sc.read_h5ad(spec.get("ad_file"))
    labels = base.obs[spec["ari_label_key"]].to_numpy()

    scores_eggfm = []
    scores_eggfm_2 = []
    scores_eggfm_3 = []
    scores_eggfm_4 = []
    scores_eggfm_5 = []
    # scores_eggfm_6 = []
    scores_pca = []
    scores_pca_2 = []
    total = 5
    run_eggfm_dimred(base, params)
    for run in range(total):
        print(f"=== Run {run+1}/{total} ===")
        qc = base.copy()
        

        # PCA → Diffmap
        sc.pp.neighbors(qc, n_neighbors=30, use_rep="X_pca")
        sc.tl.diffmap(qc, n_comps=k)
        X_diff_pca = qc.obsm["X_diffmap"][:, :k]
        qc.obsm["X_diff_pca"] = X_diff_pca

        # PCA → Diffmap → Diffmap
        sc.pp.neighbors(qc, n_neighbors=30, use_rep="X_diff_pca")
        sc.tl.diffmap(qc, n_comps=k)
        X_diff_pca_double = qc.obsm["X_diffmap"][:, :k]
        qc.obsm["X_diff_pca_x2"] = X_diff_pca_double

        # EGGFM
        X_eggfm = qc.obsm["X_eggfm"][:, :k]

        # EGGFM DM
        sc.pp.neighbors(qc, n_neighbors=30, use_rep="X_eggfm")
        sc.tl.diffmap(qc, n_comps=k)
        X_diff_eggfm = qc.obsm["X_diffmap"][:, :k]
        qc.obsm["X_diff_eggfm"] = X_diff_eggfm

        # EGGFM DM DM
        sc.pp.neighbors(qc, n_neighbors=30, use_rep="X_diff_eggfm")
        sc.tl.diffmap(qc, n_comps=k)
        X_diff_eggfm_x2 = qc.obsm["X_diffmap"][:, :k]
        qc.obsm["X_diff_eggfm_x2"] = X_diff_eggfm_x2

        # EGGFM DM DM DM
        sc.pp.neighbors(qc, n_neighbors=30, use_rep="X_diff_eggfm_x2")
        sc.tl.diffmap(qc, n_comps=k)
        X_diff_eggm_x3 = qc.obsm["X_diffmap"][:, :k]
        qc.obsm["X_diff_eggm_x3"] = X_diff_eggm_x3

        # EGGFM DM DM DM DM
        sc.pp.neighbors(qc, n_neighbors=30, use_rep="X_diff_eggm_x3")
        sc.tl.diffmap(qc, n_comps=k)
        X_diff_eggm_x4 = qc.obsm["X_diffmap"][:, :k]
        qc.obsm["X_diff_eggm_x4"] = X_diff_eggm_x4

        scores_pca.append(compute_ari(X_diff_pca, labels, k))
        scores_pca_2.append(compute_ari(X_diff_pca_double, labels, k))
        scores_eggfm.append(compute_ari(X_eggfm, labels, k))
        scores_eggfm_2.append(compute_ari(X_diff_eggfm, labels, k))
        scores_eggfm_3.append(compute_ari(X_diff_eggfm_x2, labels, k))
        scores_eggfm_4.append(compute_ari(X_diff_eggm_x3, labels, k))
        scores_eggfm_5.append(compute_ari(X_diff_eggm_x4, labels, k))

    print("\n=== Variance results ===")
    print(f"PCA→DM:    mean={np.mean(scores_pca):.4f}, std={np.std(scores_pca):.4f}")
    print(
        f"PCA→DM2:   mean={np.mean(scores_pca_2):.4f}, std={np.std(scores_pca_2):.4f}"
    )
    print(
        f"EGGFM:     mean={np.mean(scores_eggfm):.4f}, std={np.std(scores_eggfm):.4f}"
    )
    print(
        f"EGGFM DM:  mean={np.mean(scores_eggfm_2):.4f}, std={np.std(scores_eggfm_2):.4f}"
    )
    print(
        f"EGGFM DM2: mean={np.mean(scores_eggfm_3):.4f}, std={np.std(scores_eggfm_3):.4f}"
    )
    print(
        f"EGGFM DM3: mean={np.mean(scores_eggfm_4):.4f}, std={np.std(scores_eggfm_4):.4f}"
    )
    print(
        f"EGGFM DM4: mean={np.mean(scores_eggfm_5):.4f}, std={np.std(scores_eggfm_5):.4f}")
    
main()

[Energy DSM] Epoch 1/50  loss=150569.2366
[Energy DSM] Epoch 2/50  loss=150262.9158
[Energy DSM] Epoch 3/50  loss=150341.7201
[Energy DSM] Epoch 4/50  loss=150229.2161
[Energy DSM] Epoch 5/50  loss=150501.3685
[Energy DSM] Epoch 6/50  loss=150661.6498
[Energy DSM] Epoch 7/50  loss=150328.3106
[Energy DSM] Epoch 8/50  loss=150362.8190
[Energy DSM] Epoch 9/50  loss=150594.7663
[Energy DSM] Epoch 10/50  loss=150387.4110
[Energy DSM] Epoch 11/50  loss=150386.8015
[Energy DSM] Epoch 12/50  loss=150338.2388
[Energy DSM] Epoch 13/50  loss=150331.0066
[Energy DSM] Epoch 14/50  loss=150277.0755
[Energy DSM] Epoch 15/50  loss=150369.9458
[Energy DSM] Epoch 16/50  loss=150325.3802
[Energy DSM] Epoch 17/50  loss=150341.1927
[Energy DSM] Epoch 18/50  loss=150427.2059
[Energy DSM] Epoch 19/50  loss=150214.5641
[Energy DSM] Epoch 20/50  loss=150398.2535
[Energy DSM] Epoch 21/50  loss=150153.7875
[Energy DSM] Epoch 22/50  loss=150179.0007
[Energy DSM] Epoch 23/50  loss=150329.1311
[Energy DSM] Epoch 2

/home/machina/miniconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== Run 2/5 ===
=== Run 3/5 ===
=== Run 4/5 ===
=== Run 5/5 ===

=== Variance results ===
PCA→DM:    mean=0.2804, std=0.0026
PCA→DM2:   mean=0.2651, std=0.0019
EGGFM:     mean=0.2770, std=0.0038
EGGFM DM:  mean=0.2930, std=0.0003
EGGFM DM2: mean=0.2828, std=0.0026
EGGFM DM3: mean=0.2892, std=0.0043
EGGFM DM4: mean=0.2885, std=0.0011
